In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
df = pd.read_csv("Data/county_data_clean.csv")

In [4]:
df_states = df[df["county_fips_code"] == 0] #Dataframe for states only
df_counties = df[df["county_fips_code"] != 0] #Dataframe for counties only

In [5]:
df_counties = df_counties.set_index("name")
df_states = df_states.set_index("name")

In [6]:
df_counties = df_counties.drop(["Unnamed: 0"], axis=1)
df_states = df_states.drop(["Unnamed: 0"], axis=1)

In [7]:
df_IL = df_counties[df_counties["state_abbreviation"] == "IL"]

## Logistic Regression

In [8]:
df_IL = df_IL.dropna(axis = 1, how = "all") # Dropped completely missing columns.

In [9]:
df_IL.head()

,state_fips_code,county_fips_code,5_digit_fips_code,state_abbreviation,premature_death,premature_death_black,premature_death_hispanic,premature_death_white,poor_or_fair_health,poor_physical_health_days,poor_mental_health_days,low_birthweight,low_birthweight_black,low_birthweight_hispanic,low_birthweight_white,adult_smoking,adult_obesity,food_environment_index,physical_inactivity,access_to_exercise_opportunities,excessive_drinking,alcohol_impaired_driving_deaths,sexually_transmitted_infections,teen_births,teen_births_black,teen_births_hispanic,teen_births_white,uninsured,primary_care_physicians,ratio_of_population_to_primary_care_physicians,dentists,ratio_of_population_to_dentists,mental_health_providers,ratio_of_population_to_mental_health_providers,preventable_hospital_stays,preventable_hospital_stays_black,preventable_hospital_stays_hispanic,preventable_hospital_stays_white,mammography_screening,mammography_screening_black,mammography_screening_hispanic,mammography_screening_white,flu_vaccinations,flu_vaccinations_black,flu_vaccinations_hispanic,flu_vaccinations_white,high_school_graduation,some_college,unemployment,children_in_poverty,children_in_poverty_black,children_in_poverty_hispanic,children_in_poverty_white,income_inequality,children_in_single_parent_households,social_associations,violent_crime,injury_deaths,air_pollution_particulate_matter,drinking_water_violations,severe_housing_problems,percentage_of_households_with_high_housing_costs,percentage_of_households_with_overcrowding,percentage_of_households_with_lack_of_kitchen_or_plumbing_facilities,driving_alone_to_work,driving_alone_to_work_black,driving_alone_to_work_hispanic,driving_alone_to_work_white,long_commute_driving_alone,life_expectancy,life_expectancy_black,life_expectancy_hispanic,life_expectancy_white,premature_age_adjusted_mortality,premature_age_adjusted_mortality_black,premature_age_adjusted_mortality_hispanic,premature_age_adjusted_mortality_white,child_mortality,child_mortality_black,child_mortality_hispanic,child_mortality_white,infant_mortality,infant_mortality_black,infant_mortality_hispanic,infant_mortality_white,frequent_physical_distress,frequent_mental_distress,diabetes_prevalence,hiv_prevalence,food_insecurity,limited_access_to_healthy_foods,drug_overdose_deaths,motor_vehicle_crash_deaths,insufficient_sleep,uninsured_adults,uninsured_adults_raw,adult_population_18_64,uninsured_children,other_primary_care_providers,ratio_of_population_to_primary_care_providers_other_than_physicians,disconnected_youth,median_household_income,median_household_income_black,median_household_income_hispanic,median_household_income_white,children_eligible_for_free_or_reduced_price_lunch,residential_segregation_black_white,residential_segregation_non_white_white,homicides,firearm_fatalities,homeownership,severe_housing_cost_burden,population,p_below_18_years_of_age,p_65_and_older,p_non_hispanic_african_american,p_american_indian_and_alaskan_native,p_asian,p_native_hawaiian_other_pacific_islander,p_hispanic,p_non_hispanic_white,p_not_proficient_in_english,p_females,p_rural
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Adams County,17,1,17001,IL,7333.632965,10112.973237,NaN,7317.031386,0.135849,3.577105,3.508358,0.074490,0.127976,NaN,0.071132,0.152205,0.333,8.4,0.230,0.817698,0.204621,0.371429,367.1,27.724187,41.755889,NaN,26.748971,0.050590,0.000856,1168.035088,0.000695,1439.869565,0.002431,411.391304,5590.0,11775.0,NaN,5483.0,0.49,0.50,NaN,0.49,0.39,0.23,0.43,0.39,0.900794,0.665876,0.038124,0.179,0.356680,0.152542,0.158272,4.333693,0.310505,15.170176,346.269289,81.151574,10.7,0.0,0.111214,0.094131,0.008555,0.013018,0.826353,0.840937,NaN,0.829519,0.147,78.190834,77.199332,NaN,78.135330,361.6,474.853340,NaN,361.897721,68.017054,NaN,NaN,NaN,6.079027,NaN,NaN,NaN,0.105255,0.107662,0.113,78.5,0.115,0.031710,16.514450,10.036323,0.288115,0.061577,2334.0,37904.0,0.023415,0.001117,895.054054,0.

In [10]:
missing_perc = df_IL.isna().mean().sort_values()
missing_perc[missing_perc == 0]

state_fips_code                                                         0.0
frequent_physical_distress                                              0.0
premature_age_adjusted_mortality                                        0.0
life_expectancy                                                         0.0
long_commute_driving_alone                                              0.0
driving_alone_to_work                                                   0.0
percentage_of_households_with_lack_of_kitchen_or_plumbing_facilities    0.0
percentage_of_households_with_overcrowding                              0.0
p_females                                                               0.0
severe_housing_problems                                                 0.0
drinking_water_violations                                               0.0
air_pollution_particulate_matter                                        0.0
injury_deaths                                                           0.0
social_assoc

In [11]:
imp = IterativeImputer(max_iter=100, random_state=0)
X_test = df_IL[to_impute]
imp.fit(X_test)
print((np.round(imp.transform(X_test))))

NameError: name 'to_impute' is not defined

In [ ]:
y = df_IL["uninsured_adults"]
y = list(y)
X = df_IL.dropna().drop(["uninsured_adults", 
                         "state_abbreviation", 
                         "state_fips_code", 
                         "county_fips_code", 
                         "5_digit_fips_code"], axis = 1)
binom_glm = sm.GLM(y, X.astype(float), family=sm.families.Binomial())
binom_results = binom_glm.fit()

In [ ]:
plt.plot(y, binom_results.predict(X), 'o', alpha=0.2);

In [ ]:
missing_perc = df_IL.isna().mean().sort_values()
missing_perc[missing_perc > 0]

## Random Forest

In [ ]:
labels = np.array(df_IL['uninsured_adults'])

In [ ]:
features = df_IL.drop(["uninsured_adults", 
                      "uninsured_children", 
                      "uninsured", 
                      "state_fips_code", 
                      "county_fips_code",
                      "5_digit_fips_code", 
                      "state_abbreviation"], axis = 1) 

In [ ]:
feature_list = list(features.columns) 

In [ ]:
features = np.array(features)

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

In [ ]:
X_train = pd.DataFrame(train_features, columns = feature_list)
X_train.head()

In [ ]:
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(X_train)
train_features = np.round(imp.transform(X_train))

In [ ]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [ ]:
rf.fit(train_features, train_labels)

In [ ]:
X_test = pd.DataFrame(test_features, columns = feature_list)
X_test.head()

In [ ]:
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(X_test)
test_features = np.round(imp.transform(X_test))

In [ ]:
predictions = rf.predict(test_features)

In [ ]:
errors = abs(predictions - test_labels)

In [ ]:
errors

In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Question: 
Is mutivariate feature imputation the best way to deal with missing values? 
Is there consistency in the way values in the test and train set are imputed? 
Should randomforestregressor be used for a proportion response variable? 
Hyperparameter tuning?